# <div align="center"><b> Exploración de datos EuroSAT-RGB </b></div>

<div align="right">

<!-- [![Binder](http://mybinder.org/badge.svg)](https://mybinder.org/) -->
[![nbviewer](https://img.shields.io/badge/render-nbviewer-orange?logo=Jupyter)](https://nbviewer.org/)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

</div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻


Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:

```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:

```powershell
poetry install
Invoke-Expression (poetry env activate)
```

> 📝 <em><font color='Gray'>Nota:</font></em> Para agregar `pytorch` utilizando Poetry, se utiliza el siguiente comando:
> ```bash
> # Más info: https://github.com/python-poetry/poetry/issues/6409
> potery source add --priority explicit pytorch_gpu https://download.pytorch.org/whl/cu128 # Seleccionar la wheel adecuada para tu GPU
> poetry add --source pytorch_gpu torch torchvision 
> ```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [3]:
# Recarga automática de módulos en Jupyter Notebook
%reload_ext autoreload
%autoreload 2

import random

import matplotlib.pyplot as plt

from datasets import Dataset

# Modulos propios
from vision_transformer.config import RANDOM_SEED
from vision_transformer.dataset import load_huggingface_dataset
from vision_transformer.plots import show_image, show_image_grid

2025-06-18 23:40:20.003 | INFO     | vision_transformer.config:<module>:15 - PROJ_ROOT path is: C:\Users\Usuario\vision-transformer\vision-transformer
2025-06-18 23:40:20.003 | INFO     | vision_transformer.config:<module>:19 - Actual environment is: dev


🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [4]:
random.seed(RANDOM_SEED)

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Exploración de datos sobre el conjunto EuroSAT-RGB                                                                       |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | <small>Análisis exploratorio del proceso de *análisis de datos* del [modelo] sobre el [dataset]<br/> - *Tarea:* Clasificación<br/>- *Modelo*: `NaN`<br/> - *Dataset*: [dataset] </small>|
| **Autor** | <small>[Nombre] ([correo]) </small>                                                                                                 |

</div>

## Tabla de contenidos
1. [Carga de datos](#carga-de-datos)

## Introducción

En el trabajo final correspondiente al curso Visión por Computadora III se aborda un problema de clasificación de imágenes satelitales. El objetivo principal es comparar el desempeño de distintos modelos basados en la arquitectura Vision Transformer (ViT) y contrastarlos con al menos un modelo clásico basado en redes convolucionales. Esta comparación permite poner en práctica el fine-tuning de modelos preentrenados y, al mismo tiempo, analizar el comportamiento de distintas arquitecturas en un campo muy interesante como el de imágenes satelitales.

La propuesta busca enfocarse en un problema con objetivos claros y bien delimitados, que permita explorar tanto aspectos técnicos como conceptos actuales en el área de visión por computadora. En este caso, el interés está centrado en evaluar si las arquitecturas basadas en transformers presentan ventajas frente a los modelos convolucionales tradicionales, como se sugiere en algunos estudios recientes. Por ejemplo, el trabajo [Onboard Satellite Image Classification for Earth Observation: A Comparative Study of ViT Models](https://www.arxiv.org/pdf/2409.03901) reporta resultados positivos al aplicar ViT sobre imágenes satelitales. Por otro lado, el estudio realizado en el marco del curso CS231n de la Universidad de Stanford, [Vision Transformers for Robust Analysis of Satellite Imagery](https://cs231n.stanford.edu/2024/papers/vision-transformers-for-robust-analysis-of-satellite-imagery.pdf), presenta una visión más crítica al respecto, señalando limitaciones cuando se trabaja con datos fuera de distribución.

Cabe aclarar que este trabajo tiene un carácter exploratorio y se desarrolla con fines académicos. Si bien se toma como referencia los documentos, el propósito principal es aplicar los contenidos del curso en un caso concreto, más que validar o refutar resultados científicos previos.

Para el desarrollo se utiliza el conjunto de datos [EuroSAT](https://github.com/phelber/EuroSAT?tab=readme-ov-file), basado en imágenes del satélite Sentinel-2, perteneciente al programa Copernicus. Este dataset contiene más de 27.000 imágenes geo-referenciadas distribuidas en 10 clases, correspondientes a distintas categorías de uso del suelo. Las imágenes fueron recolectadas en 2015, por lo que son anteriores al surgimiento y la adopción generalizada de transformers en tareas de visión por computadora, lo cual agrega un marco interesante a la comparación propuesta.

Este primer notebook se enfoca en una exploración inicial del conjunto de datos, con el objetivo de comprender las características de las imágenes, la distribución de clases y otros aspectos relevantes para el preprocesamiento y el entrenamiento de los modelos.



## 1. Carga de datos <a name="carga-de-datos"></a>

```python

In [5]:
dataset = load_huggingface_dataset()

2025-06-18 23:40:32.981 | INFO     | vision_transformer.dataset:load_huggingface_dataset:422 - Cargando el dataset procesado...
2025-06-18 23:40:33.866 | INFO     | vision_transformer.dataset:load_huggingface_dataset:434 - El dataset contiene múltiples conjuntos (train, test, val). Cargando todos...


Resolving data files:   0%|          | 0/24300 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2700 [00:00<?, ?it/s]

## 2. Exploración de datos

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 24300
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 2700
    })
})


In [7]:
id2label = {id:label for id, label in enumerate(dataset['train'].features['label'].names)}
label2id = {label:id for id,label in id2label.items()}

print("Cantidad de clases:", len(id2label), "\n")
for k, v in id2label.items():
    print(f"- id {k}: {v}")

Cantidad de clases: 10 

- id 0: AnnualCrop
- id 1: Forest
- id 2: HerbaceousVegetation
- id 3: Highway
- id 4: Industrial
- id 5: Pasture
- id 6: PermanentCrop
- id 7: Residential
- id 8: River
- id 9: SeaLake


In [8]:
from transformers import ViTForImageClassification, ViTImageProcessor, Trainer, TrainingArguments
from datasets import load_dataset
from PIL import Image
import numpy as np
import torch



En este notebook se trabaja con ViT Base, el primer modelo presentado por Google en el paper [An Image is Worth 16x16 Words](https://arxiv.org/pdf/2010.11929). Este modelo marcó el inicio del uso de transformers en visión por computadora, y se toma como punto de partida para establecer una primera línea de base que luego se podrá comparar con variantes como CvT o Swin Transformer.

A continuación se incluye un esquema visual del modelo para facilitar su interpretación:

![Vision Transformer](./vit.png)

La implementación del modelo se realiza utilizando la librería [transformers](https://huggingface.co/docs/transformers/en/index) de Hugging Face. En este caso, se utiliza la versión [google/vit-base-patch16-224](https://huggingface.co/google/vit-base-patch16-224), una de las variantes originales publicadas por el equipo de Google Research.

Esta versión del modelo fue preentrenada con imágenes de tamaño 224×224, lo cual se alinea con el preprocesamiento realizado en este proyecto. Cuenta con aproximadamente 86 millones de parámetros, lo que representa un buen equilibrio entre complejidad y eficiencia para trabajar en un entorno de prueba y ajuste como el planteado en este trabajo.





In [10]:
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
processor

ViTImageProcessor {
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [34]:
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=10,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)


# 🔒 Congelar todos los parámetros del backbone ViT
for param in model.vit.parameters():
    param.requires_grad = False

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from torchvision import transforms
from PIL import Image

# Define augmentation pipeline
target_size = 224  # 224

train_augment = transforms.Compose([
    transforms.RandomApply([
        transforms.RandomRotation(15)
        ], p=0.8),
    transforms.RandomApply([
        transforms.Resize((72, 72), interpolation=Image.BICUBIC),
        transforms.RandomCrop(64, padding=0)
        ], p=0.8),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
        ], p=0.8),
    transforms.Resize((224, 224), interpolation=Image.BICUBIC),  
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

val_transform = transforms.Compose([
    transforms.Resize((target_size, target_size), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

def transform(batch, train=True):
    transform_fn = train_augment if train else val_transform
    images = [transform_fn(img) for img in batch["image"]]
    return {"pixel_values": images, "label": batch["label"]}

In [12]:
encoded_ds = {
    "train": dataset["train"].with_transform(lambda x: transform(x, train=True)),
    "test": dataset["test"].with_transform(lambda x: transform(x, train=False)),
}

In [14]:
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
import transformers
training_args = TrainingArguments(
    output_dir="./vit-checkpoint",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    remove_unused_columns=False,  # IMPORTANTE para mantener pixel_values
    report_to="none"  # Usa "wandb" si querés loguear allí
)


In [40]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["test"],
    processing_class=processor,
    compute_metrics=compute_metrics,
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.798000,0.756042,0.801111
2,0.564100,0.550895,0.857037
3,0.555200,0.473859,0.878889
4,0.552000,0.444340,0.881481
5,0.430900,0.433027,0.885556


TrainOutput(global_step=7595, training_loss=0.6876554095172192, metrics={'train_runtime': 4339.1782, 'train_samples_per_second': 28.001, 'train_steps_per_second': 1.75, 'total_flos': 9.415951827351552e+18, 'train_loss': 0.6876554095172192, 'epoch': 5.0})

In [42]:
trainer.save_model("C:/Users/Usuario/vision-transformer/vision-transformer/models/vit-base")


In [43]:
# Cargar modelo y processor desde la ruta guardada
model = ViTForImageClassification.from_pretrained(
    r"C:\Users\Usuario\vision-transformer\vision-transformer\models\vit-base"
)

processor = ViTImageProcessor.from_pretrained(
    r"C:\Users\Usuario\vision-transformer\vision-transformer\models\vit-base"
)

In [44]:
# Descongelar todo el backbone ViT
for param in model.vit.parameters():
    param.requires_grad = True


In [45]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./vit-checkpoint/checkpoints/full-finetune",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    remove_unused_columns=False,
    fp16=True,
    learning_rate=5e-6,
    logging_dir="./vit-checkpoint/logs/full",
    report_to="none"
)


In [46]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["test"],
    tokenizer=processor,
    compute_metrics=compute_metrics
)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13984\3625374545.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.230600,0.099069,0.967407
2,0.080500,0.069320,0.977407
3,0.070200,0.067130,0.978148
4,0.058800,0.054856,0.980370
5,0.052000,0.054520,0.980741


TrainOutput(global_step=3800, training_loss=0.0911558010703639, metrics={'train_runtime': 22678.7011, 'train_samples_per_second': 5.357, 'train_steps_per_second': 0.168, 'total_flos': 9.415951827351552e+18, 'train_loss': 0.0911558010703639, 'epoch': 5.0})

In [47]:
trainer.save_model("C:/Users/Usuario/vision-transformer/vision-transformer/models/vit-base-finetunned")

In [11]:
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=10,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
    output_dir="./vit-checkpoint/checkpoints/new-finetune",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    remove_unused_columns=False,
    fp16=True,
    learning_rate=5e-5,
    logging_dir="./vit-checkpoint/logs/new",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["test"],
    tokenizer=processor,
    compute_metrics=compute_metrics
)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22168\2060711825.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.229400,0.057589,0.980370
2,0.066800,0.056330,0.982222
3,0.045200,0.062767,0.980000
4,0.027000,0.045178,0.987407
5,0.016900,0.048231,0.986296


TrainOutput(global_step=3800, training_loss=0.06791659894742463, metrics={'train_runtime': 14380.7366, 'train_samples_per_second': 8.449, 'train_steps_per_second': 0.264, 'total_flos': 9.415951827351552e+18, 'train_loss': 0.06791659894742463, 'epoch': 5.0})

In [17]:
trainer.save_model("C:/Users/Usuario/vision-transformer/vision-transformer/models/vit-base-normal")

In [9]:
# Cargar modelo y processor desde la ruta guardada
model = ViTForImageClassification.from_pretrained(
    r"C:\Users\Usuario\vision-transformer\vision-transformer\models\vit-base-normal"
)

processor = ViTImageProcessor.from_pretrained(
    r"C:\Users\Usuario\vision-transformer\vision-transformer\models\vit-base-normal"
)

In [22]:
import evaluate
import numpy as np
import torch

clf_metrics = evaluate.combine([
    evaluate.load("accuracy"),
    evaluate.load("f1", average="weighted"),
    evaluate.load("precision", average="weighted"),
    evaluate.load("recall", average="weighted")
])

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convertimos a numpy de forma segura
    if isinstance(logits, torch.Tensor):
        logits = logits.detach().cpu().numpy()
    elif isinstance(logits, list):
        logits = np.array([l.detach().cpu().numpy() if isinstance(l, torch.Tensor) else l for l in logits])
    elif not isinstance(logits, np.ndarray):
        logits = np.array(logits)

    predictions = np.argmax(logits, axis=-1)
    return clf_metrics.compute(predictions=predictions, references=labels)


In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    tokenizer=processor,
    compute_metrics=compute_metrics
)

results = trainer.evaluate(eval_dataset=encoded_ds["test"])
print(results)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_26128\473951198.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyError: 'image'

In [19]:
print(encoded_ds["test"][0].keys())  # debería ser ['pixel_values', 'label']



dict_keys(['pixel_values', 'label'])
